# Example notebook for performing data discovery using D3L

The D3L package is based on the ICDE 2020 paper with the same name: https://arxiv.org/pdf/2011.10427.pdf

In [ ]:
import numpy as np
from d3l.indexing.similarity_indexes import NameIndex, FormatIndex, ValueIndex, EmbeddingIndex, DistributionIndex
from d3l.input_output.dataloaders import PostgresDataLoader, CSVDataLoader
from d3l.querying.query_engine import QueryEngine
from d3l.utils.functions import pickle_python_object, unpickle_python_object

## Data indexing

There are two main steps in D3L: indexing and searching. 
Data indexing reads all datasets from a given data lake and indexes their columns using multiple LSH indexes and multiple types of similarity evidence.

### Data loading

This example cell uses a CSV data loader that requires a root path where all CSV files to index can be found. In addition Pandas-specific CSV reading arguments can be passed, such as the separator character, encoding, etc.

Other supported data loader types include Postgresql data loaders.

In [ ]:
# Postgres data loader

# dataloader = PostgresDataLoader(
#     db_host="localhost",
#     db_password="postgres",
#     db_username="postgres",
#     db_name="postgres",
#     db_port=5432
# )

In [ ]:
# CSV data loader
dataloader = CSVDataLoader(
    root_path='<path/to/the/CSV/root/>',
    sep=','
)

### Index creation
D3L uses five types of column similarity evidence and, therefore, five LSH indexes: name-based, format-based, value-based, embedding-based, and distribution-based. The last index type is characteristic to numerical columns and performs LSH indexing using the random projections hash function applied on the density-based histogram of values. This is different from the numerical approach proposed in the paper.

Note that the embedding index requires to downloading and loading into memory the pretrained FastText embedding model. This will be downloaded once in the working directory. The default gzipped English Common Crawl FastText model has 4.2 GB and its unzipped version has 6.7 GB.

In [ ]:
# Create new indexes
name_index = NameIndex(dataloader=dataloader)
pickle_python_object(name_index, './name.lsh')
print("Name: SAVED!")

format_index = FormatIndex(dataloader=dataloader)
pickle_python_object(format_index, './format.lsh')
print("Format: SAVED!")

value_index = ValueIndex(dataloader=dataloader)
pickle_python_object(value_index, './value.lsh')
print("Value: SAVED!")

embedding_index = EmbeddingIndex(dataloader=dataloader)
pickle_python_object(embedding_index, './embedding.lsh')
print("Embedding: SAVED!")

distribution_index = DistributionIndex(dataloader=dataloader)
pickle_python_object(distribution_index, './distribution.lsh')
print("Distribution: SAVED!")

In [ ]:
# Or load them from disk if they have been created
name_index = unpickle_python_object('./name.lsh')
format_index = unpickle_python_object('./format.lsh')
value_index = unpickle_python_object('./value.lsh')
embedding_index = unpickle_python_object('./embedding.lsh')
distribution_index = unpickle_python_object('./distribution.lsh')

## Data searching

Given a query table, each of the columns will be added to the relevant indexes and a top-k nearest neighbor search is performed for each query column on each index. The results are then aggregated as discussed in the paper, except for the aggregation of the five similarity scores which, in this example, is performed by a simple mean.

If no aggregation is passed, each result will consist of a (result table name, five similarity scores) tuple. Otherwise, each result will consist of a (result table name, aggregated similarity score value) tuple. In the first case, the order of the score types corresponds to the order of the indexes passed to the query engine object.

In [ ]:
qe = QueryEngine(name_index, format_index, value_index, embedding_index, distribution_index)
results = qe.table_query(table=dataloader.read_table(table_name='TEgbBqq_Food_Hygiene_Data'),
                         aggregator=None,
                         k=10)
print(results)